In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import os
import shutil 
from datetime import datetime 
import numpy as np
import copy
from hijri_converter import convert

In [2]:
gregorian_months = ['January',
                'February',
                'March',
                'April',
                'May',
                'June',
                'July',
                'August',
                'September',
                'October',
                'November',
                'December']

islamic_months = ["Muharram", "Safar", "Rabi al Awwal", 
                  "Rabi ath Thani", "Jamada al Ula", 
                  "Jumada ath Thaniyah", "Rajab", "Shaban", 
                  "Ramadan", "Shawal", "Dhu al Qa'dah", "Dhu al Hijah"]


In [3]:
df = pd.read_csv('Data\multi_feature_sales_data.csv')


In [4]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df.set_index('Date', inplace=True)

In [5]:
df_weekly = df.resample('W').agg({'Sales':'sum', 'Average Discount': 'mean', 'eid ul fitr':'mean', 'eid ul Adha':'mean', 'Wheat Harvest':'mean', 'Cotton Harvest':'mean'})


In [6]:
for c in gregorian_months:
    df_weekly[c] = 0
for c in islamic_months:
    df_weekly[c] = 0


In [7]:
for index, row in df_weekly.iterrows():
    gregorian_month_name = gregorian_months[index.month-1]

    islamic_month_index = convert.Gregorian.fromdate(index).to_hijri().month - 1
    islamic_month_name = islamic_months[islamic_month_index - 1]
    
    df_weekly.at[index, gregorian_month_name] = 1
    df_weekly.at[index, islamic_month_name] = 1

In [8]:
df_weekly['Week 1'] =0
df_weekly['Week 2'] =0
df_weekly['Week 3'] =0
df_weekly['Week 4'] =0
df_weekly['Week 5'] =0

In [11]:
for index, row in df_weekly.iterrows():
    week_number = index.isocalendar()[1] - index.replace(day=1).isocalendar()[1] + 1
    print(week_number)
    if week_number == 1:
        df_weekly.at[index, 'Week 1'] = 1
        
    elif week_number == 2:
        df_weekly.at[index, 'Week 2'] = 1

    elif week_number == 3:
        df_weekly.at[index, 'Week 3'] = 1
        
    elif week_number == 4:
        df_weekly.at[index,'Week 4'] = 1
        
    elif week_number == 5:
        df_weekly.at[index, 'Week 5'] = 1
        

1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
-51
-50
-49
-48
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
1
2
3
4
5
1
2
3
4
1


In [12]:
df_weekly

,Sales,Average Discount,eid ul fitr,eid ul Adha,Wheat Harvest,Cotton Harvest,January,February,March,April,...,Shaban,Ramadan,Shawal,Dhu al Qa'dah,Dhu al Hijah,Week 1,Week 2,Week 3,Week 4,Week 5
Date,,,,,,,,,,,,,,,,,,,,,
2017-04-02,28088,10.500000,0.0,0.0,1.0,0.000000,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2017-04-09,73471,0.000000,0.0,0.0,1.0,0.000000,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2017-04-16,29785,0.000000,0.0,0.0,1.0,0.000000,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2017-04-23,59785,0.000000,0.0,0.0,1.0,0.000000,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2017-04-30,35325,0.000000,0.0,0.0,1.0,0.000000,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-07,148194,20.000000,0.0,0.0,0.0,1.000000,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2021-11-14,119808,18.571429,0.0,0.0,0.0,1.000000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2021-11-21,155453,21.428571,0.0,0.0,0.0,1.000000,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
df_weekly['Year'] = str(df_weekly.index.year)

In [15]:
df_weekly.to_csv('multi_feature_sales_data_weekly.csv')